In [ ]:
!pip install lightkurve

Para descargar las curvas de los exoplanetas descubiertos por TESS vamos ayudarnos con la librería astronómica lightkurve. Nos hemos descargado una lista de exoplanetas confirmados de la web de exoplanet archive (https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS&constraint=default_flag=1&constraint=disc_facility+like+%27%25TESS%25%27) y utilizando el identificador tic_id buscamos sus curvas de luz usando lightkurve.

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from tqdm import tqdm
from astropy.stats import sigma_clip
from google.colab import drive
import pickle
import warnings
import lightkurve as lk

drive.mount('/content/drive')
base_path = '/content/drive/My Drive/TFM_b'
output_file = os.path.join(base_path, 'tess_exoplanets_data.pkl')

def normalize_and_clean(time, flux, flux_err, quality, sigma=5):
    valid = np.isfinite(time) & np.isfinite(flux) & np.isfinite(flux_err)
    time, flux, flux_err, quality = time[valid], flux[valid], flux_err[valid], quality[valid]

    if len(flux) == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])

    median_flux = np.median(flux)
    if median_flux == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])

    norm_flux = flux / median_flux
    norm_flux_err = flux_err / median_flux

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        mask = sigma_clip(norm_flux, sigma=sigma, maxiters=5).mask

    return time[~mask], norm_flux[~mask], norm_flux_err[~mask], quality[~mask]

# Cargo la lista de exoplanetas confirmados por TESS descargada de NASA Exoplanet Archive
exoplanets_df = pd.read_csv(os.path.join(base_path, 'exoplanetas_tess.csv'))

all_data = []

for _, row in tqdm(exoplanets_df.iterrows(), total=len(exoplanets_df)):
    try:
        # Busca la curva de luz para este exoplaneta
        search_result = lk.search_lightcurve(f"{row['tic_id']}", mission='TESS')
        if len(search_result) == 0:
            print(f"No se encontraron datos para TIC {row['tic_id']}")
            continue

        # Cojemos solo la primera curva de luz (1)
        lc = search_result[0].download()

        time = np.array(lc.time.value)
        flux = np.array(lc.flux.value)
        flux_err = np.array(lc.flux_err.value)
        quality = np.array(lc.quality.value)

        time, norm_flux, norm_flux_err, quality = normalize_and_clean(time, flux, flux_err, quality)

        if len(time) > 0:
            star_data = {
                'ticid': row['tic_id'],
                'time': time,
                'norm_flux': norm_flux,
                'norm_flux_err': norm_flux_err,
                'quality': quality,
                'tstart': np.min(time),
                'tstop': np.max(time),
                'tessmag': row.get('tessmag', 0),
                'teff': row.get('teff', 0),
                'radius': row.get('radius', 0),
                'planet_name': row.get('planet_name', '')
            }
            all_data.append(star_data)

    except Exception as e:
        print(f"Error processing {row['tic_id']}: {str(e)}")
        import traceback
        traceback.print_exc()

# Guardamos los datos en otro fichero pickle
with open(output_file, 'wb') as f:
    pickle.dump(all_data, f)

print(f"Datos de exoplanetas guardados en {output_file}")
print(f"Total de exoplanetas procesados con éxito: {len(all_data)}")

(1) - Para un exoplaneta confirmado es común que haya muchas curvas de luz correspondientes a diferentes observaciones de diferentes sectores realizadas para confirmar su existencia. De este modo, los objetos exoplaneta tendrán muchos más puntos que los objetos no exoplaneta. Para no introducir ruido en el futuro algoritmo y evitar que aprenda que los objetos con más observacioens son exoplanetas, se descarga solo una de las curvas de luz de exoplanetas, en concreto, la más reciente ya que lightkurve ordena las más recientes primero.

In [ ]:
with open('/content/drive/My Drive/TFM_b/tess_exoplanets_data.pkl', 'rb') as f:
    data = pickle.load(f)
    
print(f"Número total de objetos: {len(data)}")
unique_ticids = len(set(obj['ticid'] for obj in data if obj['ticid'] is not None))
print(f"Número de TICIDs únicos: {unique_ticids}")

# Mostrar información de los 10 primeros objetos
print("\nInformación de los 10 primeros objetos:")
for i, obj in enumerate(data[:10]):
    print(f"\nObjeto {i+1}:")
    print(f"TICID: {obj['ticid']}")
    print(f"Número de puntos de datos: {len(obj['time'])}")
    print(f"Rango de tiempo de observación: {obj['tstart']:.2f} - {obj['tstop']:.2f}")
    print(f"Magnitud TESS: {obj['tessmag']:.2f}" if obj['tessmag'] is not None else "Magnitud TESS: No disponible")
    print(f"Temperatura efectiva: {obj['teff']:.2f} K" if obj['teff'] is not None else "Temperatura efectiva: No disponible")
    print(f"Radio estelar: {obj['radius']:.2f} R_sol" if obj['radius'] is not None else "Radio estelar: No disponible")

num_data_points = [len(obj['time']) for obj in data]
teff_values = [obj['teff'] for obj in data if obj['teff'] is not None]
radius_values = [obj['radius'] for obj in data if obj['radius'] is not None]
tessmag_values = [obj['tessmag'] for obj in data if obj['tessmag'] is not None]

print("\nEstadísticas generales:")
print(f"Promedio de puntos de datos por objeto: {np.mean(num_data_points):.2f}")
print(f"Mediana de puntos de datos por objeto: {np.median(num_data_points):.2f}")
if teff_values:
    print(f"Rango de temperaturas efectivas: {min(teff_values):.2f} - {max(teff_values):.2f} K")
else:
    print("No hay datos válidos de temperatura efectiva")
if radius_values:
    print(f"Rango de radios estelares: {min(radius_values):.2f} - {max(radius_values):.2f} R_sol")
else:
    print("No hay datos válidos de radio estelar")
if tessmag_values:
    print(f"Rango de magnitudes TESS: {min(tessmag_values):.2f} - {max(tessmag_values):.2f}")
else:
    print("No hay datos válidos de magnitud TESS")

example_obj = data[0]
print(f"\nEjemplo de curva de luz para TICID {example_obj['ticid']}:")
print("Tiempo  Flujo Normalizado  Error de Flujo")
for t, f, e in zip(example_obj['time'][:5], example_obj['norm_flux'][:5], example_obj['norm_flux_err'][:5]):
    print(f"{t:.2f}  {f:.5f}  {e:.5f}")

Número total de objetos: 538
Número de TICIDs únicos: 436

Información de los 10 primeros objetos:

Objeto 1:
TICID: TIC 441420236
Número de puntos de datos: 17721
Rango de tiempo de observación: 1325.94 - 1353.05
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 2:
TICID: TIC 441420236
Número de puntos de datos: 17721
Rango de tiempo de observación: 1325.94 - 1353.05
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 3:
TICID: TIC 293607057
Número de puntos de datos: 15848
Rango de tiempo de observación: 2989.73 - 3014.16
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 4:
TICID: TIC 410214986
Número de puntos de datos: 18277
Rango de tiempo de observación: 1325.30 - 1353.18
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 5:
TICID: TIC 370133522
Número de puntos de datos: 17541
Rango de tiempo de observación: 1653.92 - 1682.36
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 6:
TICID: TIC 279741379
Número de puntos de datos: 18277
Rango de tiempo de observación: 1325.30 - 1353.18
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 7:
TICID: TIC 260708537
Número de puntos de datos: 18279
Rango de tiempo de observación: 1325.29 - 1353.18
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 8:
TICID: TIC 262530407
Número de puntos de datos: 12851
Rango de tiempo de observación: 1385.94 - 1406.22
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 9:
TICID: TIC 452866790
Número de puntos de datos: 16341
Rango de tiempo de observación: 1491.64 - 1516.09
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Objeto 10:
TICID: TIC 413248763
Número de puntos de datos: 13039
Rango de tiempo de observación: 1517.83 - 1542.00
Magnitud TESS: 0.00
Temperatura efectiva: 0.00 K
Radio estelar: 0.00 R_sol

Ejemplo de curva de luz para TICID TIC 441420236:
Tiempo  Flujo Normalizado  Error de Flujo
1325.94  0.99441  0.00021
1325.94  0.99445  0.00021
1325.95  0.99440  0.00021
1325.95  0.99397  0.00021
1325.95  0.99456  0.00021

Aquí vemos que se han descargado más curvas de luz que objetos hay, pese a que para cada exoplaneta solo estábamos descargando la curva de luz más reciente. Esto se debe a que varios exoplanetas pueden orbitar la misma estrella. Puesto que no hay forma sencilla de separar la curva de luz para cada exoplaneta individualmente utilizando solo el TIC ID. Por simplicidad para no complicar el ejercicio con el poco tiempo disponible se va a coger solo una curva de luz y eliminaremos los duplicados.

In [ ]:
with open('/content/drive/My Drive/TFM_b/tess_exoplanets_data.pkl', 'rb') as f:
    data = pickle.load(f)

print(f"Número total de objetos antes de eliminar duplicados: {len(data)}")

# Crear un diccionario para almacenar la primera aparición de cada TICID
unique_data = {}
for obj in data:
    ticid = obj['ticid']
    if ticid not in unique_data:
        unique_data[ticid] = obj  # Guardar la primera aparición de cada TICID

# Convertir el diccionario de vuelta a una lista
filtered_data = list(unique_data.values())

print(f"Número total de objetos después de eliminar duplicados: {len(filtered_data)}")

output_file_filtered = '/content/drive/My Drive/TFM_b/tess_exoplanets_data_filtered.pkl'
with open(output_file_filtered, 'wb') as f:
    pickle.dump(filtered_data, f)

print(f"Datos filtrados guardados en {output_file_filtered}")

Número total de objetos antes de eliminar duplicados: 538<br>
Número total de objetos después de eliminar duplicados: 436<br>
Datos filtrados guardados en /content/drive/My Drive/TFM_b/tess_exoplanets_data_filtered.pkl